# Sentinel Radiometric Terrain Correction

This notebook i based on the snappy_topsar_insar_share.py by Alaska Satellite Facility.
## Imports

In [3]:
from snappy import GPF
from snappy import ProductIO
from snappy import HashMap
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (20,10)

from time import *

## Preparation of variables

In [5]:

iDebug = True
iWriteEachStep = False

GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()
prods = []
file1 = ""
file2 = ""
strFile1 = ""
strFile2 = ""
strInSAR = ""
target1 = ""
target2 = ""
targetTOPSplit1 = ""
targetTOPSplit2 = ""
targetbackGeo = ""
targetInterferogram = ""
targetTOPSARDeburst = ""




## Definition of functions

In [6]:
def readFiles(filename1, filename2):
    global file1
    global file2
    global strFile1
    global strFile2
    global strInSAR

    # get my 2 files
    if iDebug: print (filename1)
    if iDebug: print (filename2)
    
    file1 = ProductIO.readProduct(filename1)
    if iDebug: print ("Read file: " + file1.getName())
    strFile1 = file1.getName()
    file2 = ProductIO.readProduct(filename2)
    if iDebug: print ("Read file: " + file2.getName() + ('\n'))
    strFile2 = file2.getName()

    # InSAR Stack name: grab the leading common bits, and remove the dates et al
    strFind = "20"
    strInSAR = file1.getName()
    i = file1.getName().find(strFind)
    if i: strInSAR = file1.getName()[:i]
    if iDebug: print (strInSAR)


def writeFiles(target, filename, strStep):
    ofile = "/opt/"+filename + strStep
    if iDebug: print (ofile)
    if target:
        ProductIO.writeProduct(target, ofile, 'BEAM-DIMAP')   # TO DO:  allow other types here: BEAM-DIMAP, GeoTIFF-BigTIFF, HDF5
    else:
        print ("writeFiles Error: No target given")


def tOPSARSplit():
    ### TOPSAR-Split
    global file1
    global file2
    global targetTOPSplit1
    global targetTOPSplit2

    parameters = ""
    parameters = HashMap()
    parameters.put('subswath', 'IW1')
    parameters.put('selectedPolarisations', 'VV')
    if iDebug: print ("Entering: TOPSAR-Split")

    targetTOPSplit1 = GPF.createProduct("TOPSAR-Split", parameters, file1)

    targetTOPSplit2 = GPF.createProduct("TOPSAR-Split", parameters, file2)
    if iDebug: print ("TOPSAR-Split working")


def applyOrbitFile():
    global targetTOPSplit1
    global targetTOPSplit2
    global target1
    global target2
    global prods

    if iDebug: print ("Entering: ApplyOrbit")

    parameters = HashMap()
    parameters.put("Orbit State Vectors", "Sentinel Precise (Auto Download)")
    parameters.put("Polynomial Degree", 3)
    target1 = GPF.createProduct("Apply-Orbit-File", parameters, targetTOPSplit1)
    if iDebug: print ("ApplyOrbit working: 1")


    target2 = GPF.createProduct("Apply-Orbit-File", parameters, targetTOPSplit2)
    if iDebug: print ("ApplyOrbit working: 2")
    prods.append(target2)
    prods.append(target1)
    if iDebug: print ("Added files in reverse order") 


def backGeocoding():
    ### backGeocoding
    global prods
    global targetbackGeo


    parameters = ""
    parameters = HashMap()
    parameters.put("Digital Elevation Model", "SRTM 3Sec (Auto Download)")
    parameters.put("DEM Resampling Method", "BICUBIC_INTERPOLATION")
    parameters.put("Resampling Type", "BISINC_5_POINT_INTERPOLATION")
    parameters.put("Mask out areas with no elevation", True)
    parameters.put("Output Deramp and Demod Phase", False)

    if iDebug:
        for u in prods:
            print (u)

    targetbackGeo = GPF.createProduct("Back-Geocoding", parameters, prods)
    if iDebug: print ("Back-Geocoding working")


def interferogram():
    ### Interferogram
    # print GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi('Interferogram')
    # Interferogram = jpy.get_type('org.esa.s1tbx.insar.gpf.CreateInterferogramOp')
    global targetbackGeo
    global targetInterferogram

    if iDebug: print ("Entering: Interferogram")

    parameters = ""
    parameters = HashMap()
    parameters.put("Subtract flat-earth phase", True)
    parameters.put("Degree of \"Flat Earth\" polynomial", 5)
    parameters.put("Number of \"Flat Earth\" estimation points", 501)
    parameters.put("Orbit interpolation degree", 3)
    parameters.put("Include coherence estimation", True)
    parameters.put("Square Pixel", False)
    parameters.put("Independent Window Sizes", False)   
    parameters.put("Coherence Azimuth Window Size", 10)
    parameters.put("Coherence Range Window Size", 10)
    targetInterferogram = GPF.createProduct("Interferogram", parameters, targetbackGeo)
    if iDebug: print ("Interferogram working")


def tOPSARDeburst():
    ### TOPSAR-Deburst
    global targetInterferogram
    global targetTOPSARDeburst

    if iDebug: print ("Entering: TOPSARDeburst")

    parameters = HashMap()
    parameters.put("Polarisations", "VV")
    targetTOPSARDeburst = GPF.createProduct("TOPSAR-Deburst", parameters, targetInterferogram)
    if iDebug: print ("TOPSARDeburst working")

def plotBand(product, band, vmin, vmax):
     
    band = product.getBand(band)

    w = band.getRasterWidth()
    h = band.getRasterHeight()

    band_data = np.zeros(w * h, np.float32)
    band.readPixels(0, 0, w, h, band_data)

    band_data.shape = h, w

    width = 12
    height = 12
    plt.figure(figsize=(width, height))
    imgplot = plt.imshow(band_data, cmap=plt.cm.binary, vmin=vmin, vmax=vmax)
    
    return imgplot 

def DestinationThread() :
    while True :
        f, args = q.get()
        f(*args)

## Start processing

In [7]:
# NOTE:  add them in date order, oldest to newest
filename1 = "/eodata/Sentinel-1/SAR/SLC/2017/12/21/S1A_IW_SLC__1SDV_20171221T050000_20171221T050027_019796_021AC2_6395.SAFE"
filename2 = "/eodata/Sentinel-1/SAR/SLC/2018/08/21/S1A_IW_SLC__1SDV_20180821T163602_20180821T163629_023347_028A1C_F4EC.SAFE"

print ("Snappy TOPSAR InSAR: start")

### Read in the files

readFiles(filename1, filename2)


Snappy TOPSAR InSAR: start
/eodata/Sentinel-1/SAR/SLC/2017/12/21/S1A_IW_SLC__1SDV_20171221T050000_20171221T050027_019796_021AC2_6395.SAFE
/eodata/Sentinel-1/SAR/SLC/2018/08/21/S1A_IW_SLC__1SDV_20180821T163602_20180821T163629_023347_028A1C_F4EC.SAFE
Read file: S1A_IW_SLC__1SDV_20171221T050000_20171221T050027_019796_021AC2_6395
Read file: S1A_IW_SLC__1SDV_20180821T163602_20180821T163629_023347_028A1C_F4EC

S1A_IW_SLC__1SDV_


In [8]:
### tOPSAR-Split step
tOPSARSplit()

if iWriteEachStep: 
    writeFiles(targetTOPSplit1, strFile1, "_tOPSARSplit")
    writeFiles(targetTOPSplit2, strFile2, "_tOPSARSplit")
    


Entering: TOPSAR-Split
TOPSAR-Split working


In [9]:
### ApplyOrbitFile step
applyOrbitFile()

if iWriteEachStep: 
    writeFiles(target1, strFile1, "_AppOrb")    
    writeFiles(target2, strFile2, "_AppOrb")    

Entering: ApplyOrbit
ApplyOrbit working: 1
ApplyOrbit working: 2
Added files in reverse order


In [10]:
### backGeocoding step
backGeocoding()
if iWriteEachStep:
    writeFiles(targetbackGeo, strInSAR, "_CFCoReg")    

org.esa.snap.core.datamodel.Product[name=S1A_IW_SLC__1SDV_20180821T163602_20180821T163629_023347_028A1C_F4EC_Orb]
org.esa.snap.core.datamodel.Product[name=S1A_IW_SLC__1SDV_20171221T050000_20171221T050027_019796_021AC2_6395_Orb]
Back-Geocoding working


In [11]:
### Interferogram step
interferogram()

if iWriteEachStep:
    writeFiles(targetInterferogram, strInSAR, "_Interf")    

Entering: Interferogram
Interferogram working


In [12]:


### TOPSARDeburst step  (LAST step so naming this _InSAR)
tOPSARDeburst()

writeFiles(targetTOPSARDeburst, strInSAR, "result")

print ("Snappy TOPSAR InSAR: end")



Entering: TOPSARDeburst
TOPSARDeburst working
/opt/S1A_IW_SLC__1SDV_result


RuntimeError: java.io.IOException: failed to create data output directory: /opt/S1A_IW_SLC__1SDV_result.data